In [13]:
import cv2 as cv
import numpy as np

# Resmi yüklüyoruz.
img = cv.imread('resim2.png')
output = img.copy()

# Gri tonlamalı ve bulanıklaştırılmış görüntüsünü oluşturuyoruz.
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray = cv.medianBlur(gray, 5)

# Daire tespiti kodu
circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)

if circles is not None:
    detected_circles = np.uint16(np.around(circles))

    for circle in detected_circles[0, :]:
        x, y, r = circle
        cv.circle(output, (x, y), r, (0, 0, 0), 3)  # Dairelerin rengi siyah
        cv.circle(output, (x, y), 2, (0, 0, 0), 3)  # Merkez işaretinin rengi siyah
        shift_amount = 0  # Sağa kaydırılacak miktar, yazılar üst üste binmesin.
        cX_shifted = x + shift_amount
        cv.putText(output, 'Daire', (cX_shifted, y), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)

# Kare, üçgen, beşgen ve dikdörtgen tespiti kodları.

edges = cv.Canny(gray, 50, 150)
contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

for contour in contours:
    approx = cv.approxPolyDP(contour, 0.03 * cv.arcLength(contour, True), True)
    num_sides = len(approx)

    if num_sides == 3:
        cv.drawContours(output, [contour], 0, (0, 0, 0), 1)  # Üçgenin rengi siyah
        M = cv.moments(contour)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        cv.putText(output, 'Ucgen', (cX, cY), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
    elif num_sides == 4:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if 0.95 <= aspect_ratio <= 1.05:
            cv.drawContours(output, [contour], 0, (0, 0, 0), 2)  # Kare rengi siyah
            M = cv.moments(contour)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv.putText(output, 'Kare', (cX, cY), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
        else:
            cv.drawContours(output, [contour], 0, (0, 0, 0), 2)  # Dikdörtgen rengi siyah
            M = cv.moments(contour)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv.putText(output, 'Dikdortgen', (cX, cY), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
    elif num_sides == 5:
        cv.drawContours(output, [contour], 0, (0, 0, 0), 2)  # Beşgen rengi siyah
        M = cv.moments(contour)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        cv.putText(output, 'Besgen', (cX, cY), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
    else:
        # Elips tespiti
        if len(contour) >= 5:  # Elips için en az 5 nokta gerekir
            area = cv.contourArea(contour)
            perimeter = cv.arcLength(contour, True)
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            roundness_threshold = 0.7  # Yuvarlaklık derecesi eşik değeri

            if circularity < roundness_threshold:
                (x, y), (MA, ma), angle = cv.fitEllipse(contour)
                cv.ellipse(output, ((x, y), (MA, ma), angle), (0, 0, 0), 2)  # Elips rengi siyah
                M = cv.moments(contour)
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                cv.putText(output, 'Elips', (cX, cY), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)

resized_output = cv.resize(output, (0, 0), fx=3, fy=3)

cv.imshow('Tespit Edilen Şekiller', resized_output)
cv.waitKey(0)
cv.destroyAllWindows()
